IN THIS CODE I TRY TO SOLVE REBER GRAMMAR

PART 1: BUILD THE SIMPLE AND COMPLEX GENERATORS

In [1305]:
# Import dependencies
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix

In [1306]:
# add the character before!
def rerun_short(st,go,alt):
  no_prefix=0
  while st.endswith(alt)==False:
    proba=np.random.rand(1)
    if proba>=.5:
      st+=go
    else:
      st+=alt
    no_prefix+=1
  return st[-no_prefix:]

In [1307]:
def rerun_long(st,go_inside="t",alt_inside="v"):
  begin=len(st)
  while True:
    st+="x"
    st+=rerun_short(st,go_inside,alt_inside)
    proba_=np.random.rand(1)
    if proba_>=.5:
      st+="v"
      break    
    else:
      st+="p"    
  return st[begin:] 

In [1308]:
# Reber grammar-simple generator
def simple_generator():
  z="b"
  proba_1=np.random.rand(1)
  if proba_1>=.5:
    z+="t"
    z+=rerun_short(z,"s","x")
    proba_2=np.random.rand(1)
    if proba_2>=.5:
      z+="se"
    else:
      z+=rerun_long(z)
      z+="e"
  else:
    z+="p"
    z+=rerun_short(z,"t","v")
    proba_3=np.random.rand(1)
    if proba_3>=.5: 
      z+="ve"
    else:
      z+="p"
      z+=rerun_long(z)
      z+="e"
  return z


In [1309]:
# Reber grammar-complex generator
def complex_generator():
  z="b"
  proba=np.random.rand(1)
  if proba>=.5:
    z+="t"
    z+=simple_generator()
    z+="te"
  else:
    z+="p"
    z+=simple_generator()
    z+="pe"
  return z

In [1310]:
complex_generator()

'btbtxxtvpxvvete'

PART 2: CREATING THE DATASET

In [1311]:
# Creating a fancy function for illegal words. With the permutations the newly generated set 
# is not granted to satisfy the grammar rules w.r.t. the order
np.random.seed(12)
a=simple_generator()
k=[element for element in a]
g=np.random.permutation(np.array(k))



In [1312]:
def complete_set(instances,generator):
  max=0
  for i in range(instances):
    X_true_one=np.array([element for element in generator()])
    X_false_one=np.random.permutation(X_true_one)
    if len(X_true_one)>max:
      max=len(X_true_one)
    zero_padding_one=np.zeros((max-len(X_true_one)))
    X_true_one=np.concatenate((X_true_one,zero_padding_one))
    X_false_one=np.concatenate((X_false_one,zero_padding_one))
    if i==0:
      X_trues=X_true_one[tf.newaxis,:]
      X_falses=X_false_one[tf.newaxis,:]
    else:
      X_true_one=X_true_one[tf.newaxis,:]
      X_false_one=X_false_one[tf.newaxis,:]
      if max>X_trues.shape[-1]:
        zero_padding_two=np.zeros((len(X_trues),(max-X_trues.shape[-1])))
        X_trues=np.concatenate((X_trues,zero_padding_two),axis=1)
        X_falses=np.concatenate((X_falses,zero_padding_two),axis=1)
      X_trues=np.concatenate((X_trues,X_true_one),axis=0)
      X_falses=np.concatenate((X_falses,X_false_one),axis=0)
      if i==instances-1:   # attach the labels at the end of the second axis
        True_set=np.concatenate((X_trues,np.ones((instances,1))),axis=1)
        False_set=np.concatenate((X_falses,np.zeros((instances,1))),axis=1)
        return np.concatenate((True_set,False_set),axis=0)  


In [1313]:
# NB: this cell is quite slow
n_train=5000*2
train=np.random.permutation(complete_set(5000,simple_generator))  # creating the dataset and shuffling it the first time

In [1314]:
def preprocess(dataset):
  # Since we have a list of strings, we need to tokenize those.
  distinct=np.unique(dataset)
  # Since we have only seven distinct characters, we can easily convert them manually to integers 
  new_dataset=np.empty(dataset.shape)
  new_dataset=np.reshape(new_dataset,new_dataset.shape[0]*new_dataset.shape[1])
  diction={"0.0":0,"1.0":1,"b":2,"e":3,"p":4,"s":5,"t":6,"v":7,"x":8}
  for index,element in enumerate(np.reshape(dataset,dataset.shape[0]*dataset.shape[1])):
    new_dataset[index]=diction[element]
  return tf.constant(new_dataset.reshape(dataset.shape[0],dataset.shape[1]),dtype=tf.float32)
  

In [1315]:
train=preprocess(train)

In [1316]:
valid_=np.random.permutation(complete_set(2000,simple_generator))

In [1317]:
valid_=preprocess(valid_)

STEP 3: GO TENSORFLOW

In [1318]:
train=tf.data.Dataset.from_tensor_slices(train)
for i in train.take(1):
  print(i.numpy())

[2. 6. 8. 8. 6. 7. 7. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [1319]:
batch_size=16
train=train.shuffle(500).repeat().batch(batch_size)
train=train.map(lambda item:(item[:,:-1],item[:,-1:]))
for item in train.take(1):
  print(item)  # A tuple containing features and label
train=train.prefetch(1)

(<tf.Tensor: shape=(16, 66), dtype=float32, numpy=
array([[2., 6., 5., ..., 0., 0., 0.],
       [2., 6., 8., ..., 0., 0., 0.],
       [2., 6., 8., ..., 0., 0., 0.],
       ...,
       [5., 2., 8., ..., 0., 0., 0.],
       [2., 6., 8., ..., 0., 0., 0.],
       [8., 7., 2., ..., 0., 0., 0.]], dtype=float32)>, <tf.Tensor: shape=(16, 1), dtype=float32, numpy=
array([[1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.]], dtype=float32)>)


In [1320]:
# for validation data:
valid=tf.data.Dataset.from_tensor_slices(valid_)
valid=valid.batch(batch_size)
valid=valid.map(lambda item:(item[:,:-1],item[:,-1:]))
valid.prefetch(1)

<PrefetchDataset shapes: ((None, 57), (None, 1)), types: (tf.float32, tf.float32)>

In [1321]:
def model_builder(n_units=128):
  model=tf.keras.models.Sequential([
          tf.keras.layers.Embedding((len(distinct)+1),n_units,input_shape=[None]),
          tf.keras.layers.GRU(units=n_units),
          tf.keras.layers.Dense(1,activation="sigmoid")
        ])
  return model

In [1322]:
model=model_builder()
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, None, 128)         1280      
_________________________________________________________________
gru_29 (GRU)                 (None, 128)               99072     
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 129       
Total params: 100,481
Trainable params: 100,481
Non-trainable params: 0
_________________________________________________________________


In [1323]:
opt=tf.keras.optimizers.RMSprop(lr=.01)
model.compile(loss="binary_crossentropy",optimizer=opt,metrics=["accuracy"])
history=model.fit(dataset,epochs=15,steps_per_epoch=n_train//batch_size,validation_data=valid)

Epoch 1/15
375/375 [==============================] - 3s 9ms/step - loss: 0.7111 - accuracy: 0.5000 - val_loss: 0.6859 - val_accuracy: 0.5680
Epoch 2/15
375/375 [==============================] - 3s 8ms/step - loss: 0.6701 - accuracy: 0.5893 - val_loss: 0.6830 - val_accuracy: 0.5527
Epoch 3/15
375/375 [==============================] - 3s 8ms/step - loss: 0.4413 - accuracy: 0.7525 - val_loss: 0.0279 - val_accuracy: 0.9960
Epoch 4/15
375/375 [==============================] - 3s 8ms/step - loss: 0.0736 - accuracy: 0.9837 - val_loss: 0.0333 - val_accuracy: 0.9960
Epoch 5/15
375/375 [==============================] - 3s 8ms/step - loss: 0.0334 - accuracy: 0.9950 - val_loss: 0.0343 - val_accuracy: 0.9958
Epoch 6/15
375/375 [==============================] - 3s 8ms/step - loss: 0.0175 - accuracy: 0.9960 - val_loss: 0.0269 - val_accuracy: 0.9965
Epoch 7/15
375/375 [==============================] - 3s 8ms/step - loss: 0.0192 - accuracy: 0.9973 - val_loss: 0.0304 - val_accuracy: 0.9967
Epoch 

In [1326]:
# please feed a 2d array!
inp=np.array([["b","t","s","x","x","v","v","e"]])
if model.predict(preprocess(inp))>=.5:
  print("Predicts LEGAL")
else:
  print("Predicts ILLEGAL")

  

Predicts LEGAL


In [1328]:
X_valid_hat=[]
all_preds=model.predict(valid)
for k in all_preds:
  if k>=.5:
    X_valid_hat.append(1)
  else:
    X_valid_hat.append(0)


confusion_matrix(valid_.numpy()[:,-1].astype(np.uint8),X_valid_hat)

array([[1986,   14],
       [   0, 2000]])

STEP 5: FACE THE MORE COMPLEX ONE

In [1329]:
# NB: you can drink a coffee whilst waiting this to be executed
n_train_1=7500*2
train_1=np.random.permutation(complete_set(7500,complex_generator)) 
train_1=preprocess(train_1)
valid_1_=np.random.permutation(complete_set(2000,complex_generator))
valid_1_=preprocess(valid_1_)

In [1330]:
train_1=tf.data.Dataset.from_tensor_slices(train_1)
train_1=train_1.shuffle(700).repeat().batch(batch_size)
train_1=train_1.map(lambda item:(item[:,:-1],item[:,-1:]))
train_1=train_1.prefetch(1)

In [1331]:
valid_1=tf.data.Dataset.from_tensor_slices(valid_1_)
valid_1=valid_1.batch(batch_size)
valid_1=valid_1.map(lambda item:(item[:,:-1],item[:,-1:]))
valid_1=valid_1.prefetch(1)

In [1332]:
# Let's create with a model which is similar to the one used before
def model_builder_1(n_units=128):
  model=tf.keras.models.Sequential([
          tf.keras.layers.Embedding((len(distinct)+1),n_units,input_shape=[None]),
          tf.keras.layers.GRU(units=n_units),
          tf.keras.layers.Dense(1,activation="sigmoid")
        ])
  return model

In [1333]:
model_1=model_builder_1()

In [1334]:
opt=tf.keras.optimizers.RMSprop(lr=.01)
model_1.compile(loss="binary_crossentropy",optimizer=opt,metrics=["accuracy"])
history=model_1.fit(train_1,epochs=15,steps_per_epoch=n_train_1//batch_size,validation_data=valid_1)

Epoch 1/15
375/375 [==============================] - 4s 9ms/step - loss: 0.0900 - accuracy: 0.9635 - val_loss: 0.1060 - val_accuracy: 0.9987
Epoch 2/15
375/375 [==============================] - 3s 9ms/step - loss: 0.0135 - accuracy: 0.9980 - val_loss: 0.0164 - val_accuracy: 0.9970
Epoch 3/15
375/375 [==============================] - 3s 9ms/step - loss: 0.0859 - accuracy: 0.9783 - val_loss: 0.0369 - val_accuracy: 0.9937
Epoch 4/15
375/375 [==============================] - 3s 9ms/step - loss: 0.0326 - accuracy: 0.9938 - val_loss: 0.0118 - val_accuracy: 0.9980
Epoch 5/15
375/375 [==============================] - 3s 9ms/step - loss: 0.0183 - accuracy: 0.9962 - val_loss: 0.0113 - val_accuracy: 0.9987
Epoch 6/15
375/375 [==============================] - 3s 9ms/step - loss: 0.0274 - accuracy: 0.9973 - val_loss: 0.0058 - val_accuracy: 0.9990
Epoch 7/15
375/375 [==============================] - 3s 9ms/step - loss: 0.0104 - accuracy: 0.9980 - val_loss: 0.0131 - val_accuracy: 0.9983
Epoch 

In [1335]:
inp1=np.array([["b","t","s","x","x","v","v","e"]])    # this was Legal in previous model!
inp2=np.array([["b","t","b","t","s","x","x","v","v","e","t","e"]])
for inps in (inp1,inp2):
  if model_1.predict(preprocess(inps))>=.5:
    print("Predicts LEGAL for {}".format(inps))
  else:
    print("Predicts ILLEGAL for {}".format(inps))  




Predicts ILLEGAL for [['b' 't' 's' 'x' 'x' 'v' 'v' 'e']]
Predicts LEGAL for [['b' 't' 'b' 't' 's' 'x' 'x' 'v' 'v' 'e' 't' 'e']]


In [1336]:
X_valid_hat_1=[]
all_preds_1=model_1.predict(valid_1)
for k in all_preds_1:
  if k>=.5:
    X_valid_hat_1.append(1)
  else:
    X_valid_hat_1.append(0)


confusion_matrix(valid_1_.numpy()[:,-1].astype(np.uint8),X_valid_hat_1)

array([[2000,    0],
       [   0, 2000]])